In [ ]:
%sql
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    afih.equnr AS equipamento,
    iloa.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA','GAA','GRA')
    AND SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000


In [ ]:
%sql
WITH orders_ok AS (
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih afih
    JOIN poseidon_uc.saps4hanarawview.iloa iloa
      ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
),
afih_pick AS (
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equnr,
        MAX(afih.iloan) AS iloan
    FROM poseidon_uc.saps4hanarawview.afih afih
    WHERE afih.aufnr IN (SELECT aufnr FROM orders_ok)
    GROUP BY afih.aufnr
)
SELECT
    k.aufnr AS ordem,
    k.auart AS tipo_ordem,
    k.ktext AS texto_breve,
    k.erdat AS data_criacao,
    k.bukrs AS empresa,
    k.werks AS centro_trabalho,
    p.equnr AS equipamento,
    i.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk k
JOIN orders_ok ok ON ok.aufnr = k.aufnr
LEFT JOIN afih_pick p ON p.aufnr = k.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa i ON i.iloan = p.iloan
WHERE 
    k.auart IN ('GIA','GAA','GRA')
ORDER BY k.erdat DESC, k.aufnr
LIMIT 1000



In [ ]:
%sql
-- Extração de Ordens de Manutenção (similar à IW38)
-- Filtra por tipo de ordem (GIA, GAA, GRA) e local de instalação (GHIN, GHEN, GHSM)
-- Garante uma linha por ordem (aufnr único)

WITH ordens_validas AS (
    -- Identifica ordens que possuem locais de instalação válidos
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    INNER JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
        ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN', 'GHEN', 'GHSM')
),
dados_afih AS (
    -- Seleciona dados principais do AFIH (1 registro por ordem)
    -- Em caso de múltiplos registros, pega o mais recente/relevante
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equipamento,
        MAX(afih.iloan) AS iloan,
        MAX(afih.iwerk) AS planta_manutencao,
        MAX(afih.ingpr) AS grupo_planejador,
        MAX(afih.ilart) AS tipo_atividade,
        MAX(afih.qmnum) AS numero_notificacao
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.aufnr IN (SELECT aufnr FROM ordens_validas)
    GROUP BY afih.aufnr
)
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.ernam AS criado_por,
    aufk.aedat AS data_alteracao,
    aufk.aenam AS alterado_por,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    aufk.kostl AS centro_custo,
    aufk.waers AS moeda,
    aufk.loekz AS ind_exclusao,
    afih.equipamento,
    iloa.tplnr AS local_instalacao,
    iloa.stort AS desc_local_instalacao,
    afih.planta_manutencao,
    afih.grupo_planejador,
    afih.tipo_atividade,
    afih.numero_notificacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
INNER JOIN ordens_validas AS ov
    ON ov.aufnr = aufk.aufnr
LEFT JOIN dados_afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA', 'GAA', 'GRA')  -- Filtra tipos de ordem
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000

In [ ]:
%sql
-- Extração apenas do CABEÇALHO das ordens (sem duplicatas)
-- Usando apenas a tabela AUFK para evitar multiplicação de registros
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.aedat AS data_alteracao,
    aufk.ernam AS usuario_criacao,
    aufk.aenam AS usuario_alteracao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    aufk.waers AS moeda,
    aufk.kostl AS centro_custo,
    aufk.kstar AS elemento_custo,
    aufk.objnr AS numero_objeto,
    aufk.pspel AS elemento_wbs,
    aufk.loekz AS indicador_exclusao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
WHERE
    YEAR(aufk.erdat) = 2025 AND
    aufk.auart IN ('GIA','GAA','GRA') AND
    aufk.bukrs IN ('GHIN', 'GHSM', 'EPEI')
ORDER BY aufk.aufnr
